# Accelerating End-to-End Data Science Workflows # 

## 03 - DBSCAN ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้ใช้ DBSCAN ที่เร่งความเร็วด้วย GPU เพื่อระบุกลุ่มของผู้ติดเชื้อ โดยครอบคลุมหัวข้อด้านล่างนี้:
1. [สภาพแวดล้อม](#Environment)
2. [โหลดข้อมูล](#Load-Data)
3. [การจัดกลุ่มด้วย DBSCAN](#DBSCAN-Clustering)
    * [แบบฝึกหัดที่ 1 - สร้างอินสแตนซ์ DBSCAN อีกตัว](#Exercise-#1---Make-Another-DBSCAN-Instance)
4. [แสดงภาพกลุ่มข้อมูล](#Visualize-the-Clusters)


## สภาพเเวดล้อม ##

In [ ]:
import cudf
import cuml

import cuxfilter as cxf

## โหลดข้อมูล ##
สำหรับโน้ตบุ๊กนี้ เราจะโหลดข้อมูลประชากรมาอีกครั้ง โดยเลือกเฉพาะคอลัมน์ที่เราต้องการเท่านั้น และได้มีการเพิ่มคอลัมน์ `infected` เข้ามาในข้อมูล เพื่อระบุว่าบุคคลนั้น ๆ ติดเชื้อไวรัสจำลองของเราหรือไม่

In [ ]:
gdf = cudf.read_csv('./data/pop_sample.csv', dtype=['float32', 'float32', 'float32'])
print(gdf.dtypes)
gdf.shape

In [ ]:
gdf.head()

In [ ]:
gdf['infected'].value_counts()

## การจัดกลุ่มด้วย DBSCAN ##
DBSCAN เป็นอีกหนึ่งอัลกอริทึมการจัดกลุ่มแบบไม่มีผู้สอน (unsupervised clustering algorithm) ที่มีประสิทธิภาพเป็นพิเศษเมื่อไม่ทราบจำนวนกลุ่มล่วงหน้า และกลุ่มอาจมีรูปร่างเว้าหรือรูปร่างผิดปกติอื่นๆ ซึ่งมักจะเกิดขึ้นในการวิเคราะห์เชิงภูมิสารสนเทศ (geospatial analytics)

ในชุดแบบฝึกหัดนี้ คุณจะใช้ DBSCAN เพื่อระบุกลุ่มของผู้ติดเชื้อตามตำแหน่งที่ตั้ง ซึ่งอาจช่วยให้เราสามารถระบุกลุ่มที่ได้รับเชื้อจากผู้ป่วยต้นกำเนิดเดียวกัน และช่วยในการวางแผนรับมือ

สร้างอินสแตนซ์ DBSCAN โดยใช้ `cuml.DBSCAN` กำหนดค่าอาร์กิวเมนต์ `eps` (ระยะทางสูงสุดที่จุดหนึ่งสามารถอยู่ห่างจากจุดที่ใกล้ที่สุดในคลัสเตอร์ เพื่อที่จะถือว่าอยู่ในคลัสเตอร์นั้น) ให้เป็น `5000` เนื่องจากค่า `northing` และ `easting` ที่เราสร้างขึ้นนั้นวัดเป็นเมตร นี่จะช่วยให้เราสามารถระบุกลุ่มของผู้ติดเชื้อที่แต่ละบุคคลอาจแยกจากส่วนที่เหลือของกลุ่มได้ไกลถึง 5 กิโลเมตร

ด้านล่างนี้ เราจะฝึกอัลกอริทึม DBSCAN เราเริ่มต้นด้วยการสร้าง DataFrame ใหม่จากแถวของ DataFrame ต้นฉบับที่ `infected` เป็น `1` (จริง) และตั้งชื่อว่า `infected_df` – ตรวจสอบให้แน่ใจว่าได้รีเซ็ตดัชนีของ DataFrame หลังจากนั้น ใช้ `dbscan.fit_predict` เพื่อทำการจัดกลุ่มบนคอลัมน์ `northing` และ `easting` ของ `infected_df` และเปลี่ยน Series ผลลัพธ์ให้เป็นคอลัมน์ใหม่ใน `infected_gdf` ที่ชื่อว่า "cluster" สุดท้าย ให้คำนวณจำนวนคลัสเตอร์ที่ DBSCAN ระบุ

In [ ]:
dbscan = cuml.DBSCAN(eps=5000)
# dbscan = cuml.DBSCAN(eps=10000)

infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

### Exercise #1 - สร้าง DBSCAN Instance อีกอัน ###

**คำแนะนำ**: <br>
* แก้ไขเฉพาะส่วนที่ระบุว่า `<FIXME>` เท่านั้น แล้วรันเซลล์ด้านล่างเพื่อสร้าง DBSCAN instance โดยกำหนดค่า `eps` เป็น `10000`
* แก้ไขเฉพาะส่วนที่ระบุว่า `<FIXME>` เท่านั้น แล้วรันเซลล์ด้านล่างเพื่อทำการ fit ข้อมูลและระบุกลุ่มคลัสเตอร์ที่ติดเชื้อ

In [ ]:
dbscan = cuml.DBSCAN(<<<<FIXME>>>>)

In [ ]:
infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.<<<<FIXME>>>>(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

Click ... for solution. 

## แสดงภาพกลุ่มข้อมูล

เนื่องจากเรามีชื่อคอลัมน์เหมือนกันกับตัวอย่าง K-means ได้แก่ **`easting`**, **`northing`**, และ **`cluster`** เราจึงสามารถใช้โค้ดเดียวกันเพื่อแสดงผลคลัสเตอร์ได้เลย

In [ ]:
infected_df.to_pandas().plot(kind='scatter', x='easting', y='northing', c='cluster')

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**ทำได้ดีมาก!** ไปยัง [สมุดบันทึกถัดไป](3-04_logistic_regression.ipynb) กันเลย